In [1]:
import pandas as pd
from metar import Metar
import numpy as np
from IOfuncs import *
import datetime as dt

In [5]:
def make_ml_DataFrame(taf_time, station, lat, lon, metar_path, glamp_path, hrrr_path, delay_hours = 0):
    
    
    metarDF = pd.DataFrame()
    glampDF = pd.DataFrame()
    hrrrDF = pd.DataFrame()
    
    #right now hardcoded to look at metars for 2 hours before TAF due and then predictions after
    for time in range(-6, -delay_hours, 1):
        metar_at_time = get_metar_at_time(taf_time + dt.timedelta(hours = time), metar_path).T
        metarDF[f'metar {time}'] = metar_at_time
    
    work_time = dt.timedelta(hours=-delay_hours)
    glamp_data = get_glamp_at_time(taf_time + work_time, glamp_path, station, download=True)
    hrrr_data = get_hrrr_at_time(taf_time + work_time, hrrr_path, lat, lon, download=True)
    glamp_synoptic_offset = (taf_time.hour - delay_hours) % 6 - 1
    for time in range(-delay_hours, 7, 1):
        glampDF[f'glamp {time}'] = glamp_data.iloc[time + delay_hours + glamp_synoptic_offset]
        hrrrDF[f'hrrr {time}'] = hrrr_data.iloc[time + delay_hours]
    return pd.concat([metarDF, glampDF, hrrrDF])

In [6]:
taf_time = dt.datetime(year = 2021, month = 8, day = 21, hour = 18, minute = 16)

In [7]:
make_ml_DataFrame(taf_time, 'kbos', 42.3656, -71.0096, 'Data/BOS.csv', 'Data/GLAMP data/', 'Data/hrrr/')

,metar -6,metar -5,metar -4,metar -3,metar -2,metar -1,glamp 0,glamp 1,glamp 2,glamp 3,glamp 4,glamp 5,glamp 6,hrrr 0,hrrr 1,hrrr 2,hrrr 3,hrrr 4,hrrr 5,hrrr 6
station,BOS,BOS,BOS,BOS,BOS,BOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
valid,2021-08-21 11:54,2021-08-21 12:54,2021-08-21 13:54,2021-08-21 14:54,2021-08-21 15:54,2021-08-21 16:54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tmpf,77.0,75.9,78.1,80.1,80.1,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dwpf,71.1,71.1,71.1,72.0,72.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
relh,82.03,85.09,79.1,76.36,76.36,71.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DPT_925mb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,291.5,291.5,291.5,290.2,291.5,290.8,290.2
DPT_1000mb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292.8,292.8,293.8,293.5,293.8,294.2,293.0
HGT_cloud_base,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3386.0,3058.0,1602.0,1793.0,1735.0,1927.0,2074.0
HGT_cloud_ceiling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-10000.0,3018.0,5480.0,-10000.0,-10000.0,1782.0,-10000.0
